In [ ]:
import csv
import os
import boto3
from datetime import date
import psycopg2
import requests
import json
import pandas as pd
import redshift_connector

In [ ]:
#Extract data from API
url = "https://jsearch.p.rapidapi.com/search"

querystring = {"query":"Data Engineer, Canada","page":"1","num_pages":"1","date_posted":"today"}

headers = {
	"X-RapidAPI-Key": "5c92a8003bmsh88e3a9baf2d1ca6p142d91jsn7f1ccb37e696",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

raw_data = response.json()

In [ ]:
#Connect to S3
s3 = boto3.client(
service_name = 's3',
region_name = 'ca-central-1',
aws_access_key_id = os.environ['aws_access_key_id'],
aws_secret_access_key = os.environ['aws_secret_access_key']
)

In [ ]:
#Stage raw data in S3 bucket(rawjobsdata)
s3.put_object(
    Bucket='rawjobsdata',
    Key=f"raw_jobs_data_{date.today()}.json",
    Body=json.dumps(raw_data))

In [ ]:
#transform data and store in CSV format
transformed_data_list = []
for i in raw_data["data"]:
    transformed_data_list.append({
        "employer_website": i["employer_website"],
        "job_id": i["job_id"],
        "job_employment_type": i["job_employment_type"],
        "job_title": i["job_title"],
        "job_apply_link": i["job_apply_link"],
        "job_description": i["job_description"],
        "job_city": i["job_city"],
        "job_country": i["job_country"],
        "job_posted_at_timestamp": i["job_posted_at_timestamp"],
        "employer_company_type": i["employer_company_type"]
    })

transformed_data_df = pd.DataFrame(transformed_data_list)

# Save the DataFrame as a CSV file in memory
csv_data = transformed_data_df.to_csv(index=False)

In [ ]:
#Stage transformed data in S3 bucket(transformedjobsdata)
s3.put_object(
    Bucket='transformedjobsdata',
    Key=f"transformed_jobs_data_{date.today()}.csv",
    Body=csv_data)

In [ ]:
os.environ['redshift_user'] = 'awsuser'
os.environ['redshift_password'] = 'sopInwa99'


In [ ]:
conn_params = {
    'user': os.environ['redshift_user'],
    'password': os.environ['redshift_password'],
    'host': 'redshift-cluster-2.cfz4dilotntk.ca-central-1.redshift.amazonaws.com',
    'port': 5439,
    'database': 'dev'
}


In [ ]:
conn = redshift_connector.connect(**conn_params)


In [ ]:
conn = redshift_connector.connect(
    user= os.environ['redshift_user'],
    host= 'redshift-cluster-2.cfz4dilotntk.ca-central-1.redshift.amazonaws.com',
    port= 5439,
    password= os.environ['redshift_password']
 )

conn = redshift_connector.connect(**conn_params)

